Importing necessary data

In [2]:
! kaggle datasets download -d volodymyrpivoshenko/pixel-characters-dataset
! unzip "pixel-characters-dataset.zip"

Dataset URL: https://www.kaggle.com/datasets/volodymyrpivoshenko/pixel-characters-dataset
License(s): CC-BY-SA-3.0
pixel-characters-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pixel-characters-dataset.zip
replace data/0/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Libraries, parameters and data path

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import matplotlib.pyplot as plt
data_dir = "/content/data"
img_size = 64
channels = 3
latent_dim = 128
batch_size = 64
epochs = 20000
save_interval = 1000

Data loading

In [11]:
def load_images(data_dir):
    images = []
    for folder in range(4):
        folder_path = os.path.join(data_dir, str(folder))
        if os.path.isdir(folder_path):
            for img_file in tqdm(os.listdir(folder_path)):
                img_path = os.path.join(folder_path, img_file)
                img = load_img(img_path, target_size=(img_size, img_size))
                img = img_to_array(img)
                img = (img - 127.5) / 127.5
                images.append(img)
    return np.array(images)
print("Loading dataset...")
images = load_images(data_dir)
print(f"Dataset loaded with shape: {images.shape}")


100%|██████████| 912/912 [00:00<00:00, 3065.77it/s]


Model

In [15]:
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(channels, kernel_size=4, strides=2, padding='same', activation='tanh'))
    return model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, input_shape=(img_size, img_size, channels), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model


Compiling the model

In [16]:
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')


Training the model and saving images

In [ ]:
def sample_noise(batch_size, latent_dim):
    return np.random.normal(0, 1, (batch_size, latent_dim))
def save_images(epoch, generator, examples=5):
    noise = sample_noise(examples, latent_dim)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5

    fig, axs = plt.subplots(1, examples, figsize=(15, 15))
    for i in range(examples):
        axs[i].imshow(generated_images[i])
        axs[i].axis('off')
    plt.show()
def train(epochs, batch_size, save_interval):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, images.shape[0], batch_size)
        real_imgs = images[idx]

        noise = sample_noise(batch_size, latent_dim)
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = sample_noise(batch_size, latent_dim)
        g_loss = gan.train_on_batch(noise, valid)

        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")

        if epoch % save_interval == 0:
            save_images(epoch, generator)
train(epochs, batch_size, save_interval)
